In [2]:
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn import ensemble
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [3]:
df1 = pd.read_csv('./data/PBP - 2016 - Week 1.csv')
df2 = pd.read_csv('./data/PBP - 2016 - Week 2.csv')
df3 = pd.read_csv('./data/PBP - 2016 - Week 3.csv')
df4 = pd.read_csv('./data/PBP - 2016 - Week 4.csv')
df5 = pd.read_csv('./data/PBP - 2016 - Week 5.csv')
df6 = pd.read_csv('./data/PBP - 2016 - Week 6.csv')

In [4]:
df7 = pd.read_csv('./data/PBP - 2016 - Week 7.csv')
df8 = pd.read_csv('./data/PBP - 2016 - Week 8.csv')
df9 = pd.read_csv('./data/PBP - 2016 - Week 9.csv')
df10 = pd.read_csv('./data/PBP - 2016 - Week 10.csv')

In [5]:
df = pd.concat([df1, df2, df3, df4, df5, df6])
offense_stats = pd.read_csv('./data/Offense Stats 2016.csv')
defense_stats = pd.read_csv('./data/Defense Stats 2016.csv')

df = df.dropna()

In [6]:
# test set
df_test = pd.read_csv('./data/PBP - 2016 - Week 7.csv')
df_test = df.dropna()

In [7]:
def getWinner(gameId, df):
    final = df.query('gameId == @gameId').iloc[[-1]]
    if(int(final.homeScore) > int(final.awayScore)):
        return 'home'
    else:
        return 'away'

In [11]:
# add a winner label
for index,row in df.iterrows():
    df.at[index,'winner'] = getWinner(row['gameId'], df)

In [12]:
for index,row in df_test.iterrows():
    df_test.at[index,'winner'] = getWinner(row['gameId'], df)

In [ ]:
#def getQuarterPlays(quarter, df):
#    final = df.query('quarter == @quarter')
#    return final

In [9]:
# only get first/second quarter
df = df[df.quarter < 3]

In [10]:
df_test = df_test[df_test.quarter < 3]

In [13]:
# join team data
statsJoin = df.join(offense_stats.set_index('School'), on='offenseTeam', rsuffix='_offense_stats')
statsJoin = df.join(defense_stats.set_index('School'), on='defenseTeam', rsuffix='_defense_stats')
statsJoin['TD'] = statsJoin['type'].apply(lambda x: 1 if x == 'Passing Touchdown' or x == 'Rushing Touchdown' else 0)

In [14]:
# join team data
statsJoin_test = df_test.join(offense_stats.set_index('School'), on='offenseTeam', rsuffix='_offense_stats')
statsJoin_test = df_test.join(defense_stats.set_index('School'), on='defenseTeam', rsuffix='_defense_stats')
statsJoin_test['TD'] = statsJoin['type'].apply(lambda x: 1 if x == 'Passing Touchdown' or x == 'Rushing Touchdown' else 0)

In [15]:
statsJoin = statsJoin.dropna()

In [16]:
statsJoin_test = statsJoin_test.dropna()

In [17]:
def seconds_left_in_quarter(clock):
    clock = clock.split(':')
    clock = (int(clock[0])*60) + int(clock[1])
    return clock

for index,row in statsJoin.iterrows():
    statsJoin.at[index,'clock'] = seconds_left_in_quarter(row['clock'])

In [18]:
for index,row in statsJoin_test.iterrows():
    statsJoin_test.at[index,'clock'] = seconds_left_in_quarter(row['clock'])

In [20]:
y = statsJoin[['winner']]
x = statsJoin[['homeScore',
       'awayScore', 'quarter', 'clock', 'down',
       'Rk', 'Points Allowed', 'opponent completions', 'opponent attempts',
       'opponent completion percentage', 'opponent passing yards',
       'opponent passing touchdowns', 'opponent rush attempts',
       'opponent rush yards', 'opponent average rush',
       'opponent rush touchdowns', 'total opponent plays',
       'total opponent yards', 'opponent total yards per play',
       'opponent pass first downs', 'opponent rush first downs']]

In [21]:
y_test = statsJoin_test[['winner']]
x_test = statsJoin_test[['homeScore',
       'awayScore', 'quarter', 'clock', 'down',
       'Rk', 'Points Allowed', 'opponent completions', 'opponent attempts',
       'opponent completion percentage', 'opponent passing yards',
       'opponent passing touchdowns', 'opponent rush attempts',
       'opponent rush yards', 'opponent average rush',
       'opponent rush touchdowns', 'total opponent plays',
       'total opponent yards', 'opponent total yards per play',
       'opponent pass first downs', 'opponent rush first downs']]

In [22]:
clf = MLPClassifier(solver='sgd', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(x, y)

/home/ben/.local/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [23]:
pd.DataFrame(clf.predict(x_test)).describe()

,0
count,24965
unique,1
top,home
freq,24965


In [24]:
y_test.winner.value_counts()

home    13669
away    11296
Name: winner, dtype: int64

In [ ]:
x['homeTeam']

In [ ]:
#myData=np.genfromtxt(filecsv, delimiter=",", dtype ="|a20" ,skip_header=1);

from sklearn import preprocessing
import category_encoders as ce
ce_bin = ce.BinaryEncoder(cols = ['homeTeam'])

In [ ]:
ce_bin.fit_transform(x['homeTeam'], y)